In [1]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()

data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

2020-12-27 17:49:25,137 - StockDB - DEBUG - 加载股票日线, kwargs={'filter': {'code': 'sz002132'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-27 17:49:25,186 - StockDB - DEBUG - 加载日线数据成功 size=60
2020-12-27 17:49:25,187 - StockDB - DEBUG - 加载股票信息, kwargs={'filter': {'code': 'sz002132'}, 'projection': ['name']} ...
2020-12-27 17:49:25,215 - StockDB - DEBUG - 加载股票信息成功 size=1
2020-12-27 17:49:25,219 - FundDB - DEBUG - 加载场内基金日线数据, kwargs={'filter': {'code': '159949'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-27 17:49:25,503 - FundDB - DEBUG - 加载场内基金日线数据,成功, size=60
2020-12-27 17:49:25,504 - FundDB - DEBUG - 加载基金列表, kwargs={'filter': {'code': '159949'}, 'projection': ['short_name']}
2020-12-27 17:49:25,518 - FundDB - DEBUG - 加载基金列表成功, size=1


In [15]:
# data = data_fund

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]


def kline_tooltip_fmt_func():
    return JsCode("""function(obj){
                function tips_str(pre, now, tag, field, unit) {
                    var tips = tag + ':&nbsp;&nbsp;&nbsp;&nbsp;';
                    var span = '';
                    
                    if (field == 'open' || field == 'close' ||
                        field == 'low' || field == 'high') {
                        var fixed = 3;
                        if (now['code'].startsWith('s')) {
                            fixed = 2;
                        }
                        if (pre != null) {
                            var chg = (now[field] - pre['close']).toFixed(fixed);
                            var rate = (chg * 100 / pre['close']).toFixed(2);

                            if (rate >= 0) {
                                span = '<span style="color: #F11300;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            } else {
                                span = '<span style="color: #00A800;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            }

                        } else {
                            span = now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + rate + ')<br>';
                        }
                    } else {
                        if (field == 'volume') {
                            span = (now[field] / 1000000.0).toFixed(2) + '&nbsp;万手<br>';
                        }
                        if (field == 'turnover') {
                            span = now[field] + '%<br>'
                        }
                    }
                    return tips + span;

                }
                
                var pre_data = null;
                var now_data = kdata[trade_date[obj.dataIndex]];
                if (obj.dataIndex > 0) {
                    pre_data = kdata[trade_date[obj.dataIndex-1]];
                }
                var title = now_data['code'] + '&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '<br><br>';

                if ('name' in now_data) {
                    title = now_data['name'] + '(' + now_data['code'] + ')&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '</div><br>';
                }
                
                return title + 
                    tips_str(pre_data, now_data, '开盘', 'open') + 
                    tips_str(pre_data, now_data, '最高', 'high') + 
                    tips_str(pre_data, now_data, '最低', 'low') + 
                    tips_str(pre_data, now_data, '收盘', 'close') + 
                    tips_str(pre_data, now_data, '成交', 'volume') + 
                    tips_str(pre_data, now_data, '换手', 'turnover');
                 }""")

def kline_orig_data(data):
    origData = data[:]
    origData['trade_date'] = origData['trade_date'].apply(lambda d: d.strftime('%Y-%m-%d'))
    origData.index = origData['trade_date']
    origData.fillna('-', inplace=True)
    return origData.to_dict('index')

kline = Kline()
kline.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
kline.add_js_funcs('var trade_date={}'.format(trade_date))
kline.add_xaxis(trade_date)
kline.add_yaxis('日线', k_data,
                itemstyle_opts=opts.ItemStyleOpts(
                    color='#F11300',
                    color0='#00A800',
                ),
#                markline_opts=opts.MarkLineOpts(
#                    data=[opts.MarkLineItem(type_="max", value_dim="highest")])
                tooltip_opts=opts.TooltipOpts(
                    formatter=kline_tooltip_fmt_func())
               )
kline.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

ma10 = MA(data['close'], timeperiod=10)
ma10 = [round(v, 3) for v in ma10]

wma = WMA(data['close'], timeperiod=2)

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('ma10', ma10, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')


#scatter = Scatter()
#scatter.add_xaxis(trade_date)
#scatter.add_yaxis('买入', [data.iloc[30]['high']], )
#line.render_notebook()

kline.overlap(line)
kline.render_notebook()
#

In [7]:
STOCHRSI (data)


,fastk,fastd
59,NaN,NaN
58,NaN,NaN
57,NaN,NaN
56,NaN,NaN
55,NaN,NaN
54,NaN,NaN
53,NaN,NaN
52,NaN,NaN
51,NaN,NaN
50,NaN,NaN
